# Estimate covariance matrix of financial time series (FTS)

In [1]:
import re
import sys
import warnings
import numpy as np # type: ignore
import pandas as pd # type: ignore

from scipy.linalg import eigh # type: ignore

sys.path.append('../modules')
import misc_functions as mf # type: ignore
import estimate_market_factors as emf # type: ignore
import get_financial_time_series as get_fts # type: ignore

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

## Global variables

In [2]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
output_path = "../output_files"
input_generation_date = "2024-06-24"

## Load data and filter duplicates by symbol and date

In [3]:
# Standard and Poor's 500 - S&P 500 (United States)
df_stock_index = pd.read_pickle("{}/df_sp500_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))
df_stock_index.drop_duplicates(subset = ["date", "symbol"], keep = "first", inplace = True, ignore_index = True)

## Construct covariance matrix

In [4]:
df_cov = get_fts.estimate_covariance_stock_index(df = df_stock_index, normalized = True)

## Apply Bouchaud clipping filter

In [5]:
pd.DataFrame({"eig" : eigh(df_cov)[0]}).value_counts(dropna = False, sort = True).reset_index()

,eig,count
0,-5.269294,1
1,0.590273,1
2,0.620512,1
3,0.619429,1
4,0.614335,1
...,...,...
492,0.283583,1
493,0.281739,1
494,0.279345,1
495,0.277749,1


In [6]:
p_i = df_cov.shape[0] # Number of time series (shares in stock index)
q = 1/2
m = 114
step = 8
n_i = int((1/q)*p_i) # Length of time series
delta = 20
k1_k0 = 8

In [7]:
pd.DataFrame({"eig" : eigh(emf.clipping_covariance_matrix(covariance_matrix = df_cov, n = n_i)[1])[0]}).value_counts(dropna = False, sort = True).reset_index()

,eig,count
0,0.000475,7
1,0.000475,7
2,0.000475,7
3,0.000475,7
4,0.000475,7
...,...,...
240,0.000475,1
241,0.000475,1
242,0.000475,1
243,0.000475,1


In [8]:
df_tracy_widom = pd.read_csv("{}/tracy_widom.csv".format(input_path_data_dictionary), low_memory = False) 
print(emf.estimate_tracy_widom_probability(df_tracy_widom = df_tracy_widom, z_score = 5.01))
print(emf.estimate_wishart_order_2(p = 10, n = 10, df_tracy_widom = df_tracy_widom, lambda_1 = 3))
print(emf.get_market_factors(df_tracy_widom, eigen_values = np.array([179, 190, 0.26, 3.07, 2.6]), n = 10, alpha = 0.01))

0.0
0.999838414979975
3


In [9]:
df_onatski = pd.read_csv("{}/onatski.csv".format(input_path_data_dictionary), low_memory = False)
df_onatski

,0,1,2,3,4,5,6,7,8
0,15,2.75,3.62,4.15,4.54,4.89,5.20,5.45,5.70
1,10,3.33,4.31,4.91,5.40,5.77,6.13,6.42,6.66
2,9,3.50,4.49,5.13,5.62,6.03,6.39,6.67,6.92
3,8,3.69,4.72,5.37,5.91,6.31,6.68,6.95,7.25
4,7,3.92,4.99,5.66,6.24,6.62,7.00,7.32,7.59
5,6,4.20,5.31,6.03,6.57,7.00,7.41,7.74,8.04
6,5,4.52,5.73,6.46,7.01,7.50,7.95,8.29,8.59
7,4,5.02,6.26,6.97,7.63,8.16,8.61,9.06,9.36
8,3,5.62,6.91,7.79,8.48,9.06,9.64,10.11,10.44
9,2,6.55,8.15,9.06,9.93,10.47,11.27,11.75,12.13


In [12]:
# Onatski statistical test ----
def estimate_onatski_statistic(data, k0, k1=7, index=10):
    # Splitting matrix for construct Gaussian unitary ensemble (GUE)
    data_1 = data.iloc[:int(data.shape[0] / 2),:]
    data_2 = data.iloc[int(data.shape[0] / 2):,:]
    X = data_1.values + data_2.values * 1j
    gue = pd.DataFrame((1 / data_1.shape[0]) * X.conjugate().T@X)
    eigenvalues_gue, eigenvectors_gue = eigh(gue)

    # Reorder, scales, and center the eigenvalues
    delta_k = k1 - k0 + 1
    eigenvalues = np.zeros([index])
    for i in range(index):
        # R is invariant under change of scale and center
        eigenvalues[i] = (eigenvalues_gue[data_1.shape[0] - i - 1] - 2) * np.power(data_1.shape[0], -2.0 / 3)
    
    # R statistic
    R = np.zeros([delta_k])
    j = 0
    for i in range(k0, k1 + 1):
        statistics = (eigenvalues[i] - eigenvalues[i+1])/(eigenvalues[i+1] - eigenvalues[i+2])
        
        if i==k0:
            R[j] = statistics
        else:
            R[j] = max(R[ii-1],statistics)
        j+=1
    return(max(R))

def test_onatski(TABLA, j=0, level = 1):
    ### available levels: 1,2,3,4,5,6,7,8,9,10,15
    logical = TABLA[str(j)].values > significancias_onatski.loc[level].astype(float).values
    k = 0
    for value in logical:
        if  value==False:
            number = k
            break
        k+=1
        if k==8:
            number = 8
    return(number)

In [13]:
## MODELO DE SHARPE
def estimate_sharpe_model(returns, M):
  dic={}
  bs={}
  for col in returns.columns:
    x=returns[col]
    lr=stats.linregress(M,x)
    a=lr.intercept
    b=lr.slope
    S=x-a-b*M
    dic[col]=S
    bs[col]=b
  df=pd.DataFrame(data=dic)
  df = df-df.mean(axis=0)
  df = df/df.std(axis=0)
  return df, bs

## Save data in input files for no reprocessing

In [14]:
#df_market.to_pickle("{}/df_sp500_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))